# Test File for Testing DCT2 and DCT3 Algorithms

In [1]:
%reset -f
import gc
gc.collect()

0

In [2]:
import numpy as np
from scipy.fftpack import dct, idct
import tensorflow as tf

np.random.seed(42)

n = 16
q = 5
num_samples = 3

dataset = np.random.randint(0, q, size=(num_samples, n))
print(dataset)

[[3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
 [3 4 0 3 1 4 3 0 0 2 2 1 3 3 2 3]
 [3 0 2 4 2 4 0 1 3 0 3 1 1 0 1 4]]


# **DCT-II**

### Brute Force DCT2

\begin{equation}
    c[k] = \alpha[k] \sum_{n=0}^{N-1} x[n] \cdot \cos\left( \frac{\pi (2n+1) k}{2N} \right), \quad 0 \leq k \leq N-1
    \\
    \text{where } \alpha[0] = \sqrt{\frac{1}{N}}, \text{ and } \alpha[k] = \sqrt{\frac{2}{N}} \text{ for }  k \in \{ 1, 2, \dots , N-1\}
\end{equation}

In [3]:
def dct_ii(x):
    N = len(x)
    v = np.zeros(N)

    for k in range(N):
        alpha_k = np.sqrt(1/N) if k == 0 else np.sqrt(2/N)
        summation = sum(x[n] * np.cos(np.pi * (2 * n + 1) * k / (2 * N)) for n in range(N))
        v[k] = alpha_k * summation

    return np.round(v,4)

### Recursive Radix-2 DCT-II

DCT-II Structure
$$
C^{II}_n = P^T_n
\begin{bmatrix}
I_{\frac{n}{2}} & 0 \\
0 & B_{\frac{n}{2}}
\end{bmatrix}
\begin{bmatrix}
C^{II}_{\frac{n}{2}} & 0 \\
0 & C^{II}_{\frac{n}{2}}
\end{bmatrix}
\begin{bmatrix}
I_{\frac{n}{2}} & 0 \\
0 & [W_c]_{\frac{n}{2}}
\end{bmatrix}
\bar{H}_n
$$

Bidiagonal Matrix

$$
B_{\frac{n}{2}} =
\begin{bmatrix}
\sqrt{2} & 1 &  &  &  \\
 & 1 & 1 &  &  \\
 &  & \ddots & \ddots &  \\
 &  &  & \ddots & 1 \\
 &  &  &  & 1
\end{bmatrix}
$$

Diagonal (weight) matrix

$$
[W_c]_{\frac{n}{2}} = \operatorname{diag} \left[ \left( \frac{\sec\left(\frac{(2k-1)\pi}{2n}\right)}{2} \right) \right]_{k=1}^{\frac{n}{2}}
$$

Orthogonal matrix (Hadarmard)

$$
\overline{H}_n = \frac{1}{\sqrt{2}}
\begin{bmatrix}
I_{\frac{n}{2}} & \tilde{I}_{\frac{n}{2}} \\
I_{\frac{n}{2}} & -\tilde{I}_{\frac{n}{2}}
\end{bmatrix}
$$

In [4]:
def cos2(x, n):
    if n == 2:
        H_2 = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
        return H_2 @ x
    else:
        # recursive case: n >= 4
        n1 = n // 2

        # Orthogonal matrix (Hadamard-like)
        I_half = np.eye(n1)
        I_tilde_half = np.flip(I_half, axis=1)
        H = np.block([
            [I_half, I_tilde_half],
            [I_half, -I_tilde_half]
        ])
        H_n = (1 / np.sqrt(2)) * H
        u = H_n @ x

        # Diagonal weight matrix [W_c] - matrix 3
        W_entries = [1] * n1 + [ (1 / np.cos((2*k - 1)*np.pi / (2*n))) / 2 for k in range(1, n1+1) ]
        W_c = np.diag(W_entries)
        v = W_c @ u

        # Recursive DCT-II on halves - matrix 2
        z1 = cos2(v[:n1], n1)
        z2 = cos2(v[n1:], n1)

        # Bidiagonal matrix
        B = np.eye(n1)
        np.fill_diagonal(B[:-1, 1:], 1)
        B[0, 0] = np.sqrt(2)
        B_c = np.block([
            [np.eye(n1), np.zeros((n1, n1))],
            [np.zeros((n1, n1)), B]
        ])
        w = B_c @ np.concatenate([z1, z2])

        y_my = np.concatenate([w[::2], w[1::2]])
        # print("\ny_my: ", y_my)
        # Permutation (bit-reversal-like)
        perm = np.arange(n).reshape(2, -1).T.flatten()
        # print("\nperm: ", perm)
        y = w[perm]
        # print("\ny_perm: ", y)

        return y

# **DCT-III**

### Brute Force DCT3

In [5]:
def dct_iii(v):
    N = len(v)
    x = np.zeros(N)

    for n in range(N):
        summation = 0
        for k in range(N):
            alpha_k = np.sqrt(1/N) if k == 0 else np.sqrt(2/N)
            summation += alpha_k * v[k] * np.cos(np.pi * (2 * n + 1) * k / (2 * N))
        x[n] = summation

    return np.round(x, 4)

### Recursive Radix-2 DCT-III

DCT-III
$$
C_n^{III} = \overline{H}_n^T
\begin{bmatrix}
I_{\frac{n}{2}} & 0 \\
0 & [W_c]_{\frac{n}{2}}
\end{bmatrix}
\begin{bmatrix}
C_{\frac{n}{2}}^{III} & 0 \\
0 & C_{\frac{n}{2}}^{III}
\end{bmatrix}
\begin{bmatrix}
I_{\frac{n}{2}} & 0 \\
0 & B_{\frac{n}{2}}^T
\end{bmatrix}
P_n
$$

Bidiagonal Matrix

$$
B_{\frac{n}{2}} =
\begin{bmatrix}
\sqrt{2} & 1 &  &  &  \\
 & 1 & 1 &  &  \\
 &  & \ddots & \ddots &  \\
 &  &  & \ddots & 1 \\
 &  &  &  & 1
\end{bmatrix}
$$

Diagonal (weight) matrix

$$
[W_c]_{\frac{n}{2}} = \operatorname{diag} \left[ \left( \frac{\sec\left(\frac{(2k-1)\pi}{2n}\right)}{2} \right) \right]_{k=1}^{\frac{n}{2}}
$$

Orthogonal matrix (Hadarmard)

$$
\overline{H}_n = \frac{1}{\sqrt{2}}
\begin{bmatrix}
I_{\frac{n}{2}} & \tilde{I}_{\frac{n}{2}} \\
I_{\frac{n}{2}} & -\tilde{I}_{\frac{n}{2}}
\end{bmatrix}
$$

In [6]:
def cos3(x, n):
    if n == 2:
        # base case: n = 2
        H_2 = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
        y = H_2 @ x
        return y

    else:
        # recursive case: n >= 4
        n1 = n // 2

        # Step 1: Permutation P_n
        P_n = np.concatenate([x[::2], x[1::2]])

        # Step 2: Apply B_c^T
        B = np.eye(n1)
        np.fill_diagonal(B[:-1, 1:], 1)
        B[0, 0] = np.sqrt(2)

        B_c_T = np.block([
            [np.eye(n1), np.zeros((n1, n1))],
            [np.zeros((n1, n1)), B.T]
        ])
        u = B_c_T @ P_n

        # Step 3: Recursive calls for z1 and z2
        z1 = cos3(u[:n1], n1)
        z2 = cos3(u[n1:], n1)

        # Step 4: Combine results using W_c
        # n1 amount of 1s and n1 amount of Ws
        W_c = np.diag([1] * n1 + [1 / np.cos((2 * k - 1) * np.pi / (2 * n)) / 2 for k in range(1, n1 + 1)])
        w = W_c @ np.concatenate([z1, z2])

        # Step 5: Apply H_n^T
        I_half = np.eye(n1)
        I_tilde_half = np.flip(I_half, axis=1)
        H = np.block([
            [I_half, I_tilde_half],
            [I_half, -I_tilde_half]
        ])
        H_n_T = (1 / np.sqrt(2)) * H.T
        y = H_n_T @ w

        return y

### Recursive Radix-2 DCT-III without implicit matrix multiplications

In [7]:
def DCT3(x, n):
    if n == 2:
        H_2 = (1 / tf.sqrt(2.0)) * tf.constant([[1, 1], [1, -1]], dtype=tf.float32)
        y = tf.matmul(H_2, tf.transpose(x))
        return tf.transpose(y)

    else:
        n1 = n // 2

        x = tf.convert_to_tensor(x, dtype=tf.float32)

        # Permutation
        even = x[:, ::2]
        odd = x[:, 1::2]

        # B^T
        diag = tf.ones(n1, dtype=tf.float32)
        diag = tf.tensor_scatter_nd_update(diag, [[0]], [tf.sqrt(2.0)])
        super_diag = tf.ones(n1, dtype=tf.float32)

        d1 = tf.multiply(odd, diag)
        d2 = tf.multiply(tf.concat([tf.zeros_like(odd[:, :1]), odd[:, :-1]], axis=1), super_diag)
        B_n = tf.add(d1, d2)
        u = tf.concat([even, B_n], axis=1)

        # Recursive DCTIII
        z1 = DCT3(u[:, :n1], n1)
        z2 = DCT3(u[:, n1:], n1)
        recurs_out = tf.concat([z1, z2], axis=1)

        # W_c
        k = tf.range(1, n1 + 1, dtype=tf.float32)
        W_c = 1 / (2 * tf.cos((2 * k - 1) * (tf.constant(np.pi, dtype=tf.float32) / (2 * n))))
        W_n = tf.multiply(recurs_out[:, n1:], W_c)
        v = tf.concat([recurs_out[:, :n1], W_n], axis=1)

        # H_n
        out1 = v[:, :n1]
        out2 = v[:, n1:]
        y = (1 / tf.sqrt(tf.constant(2.0, dtype=tf.float32))) * tf.concat(
                [(out1 + out2), tf.reverse((out1 - out2), axis=[1])],
                axis=1
            )

        return y

# Test Cases

In [8]:
for i, message in enumerate(dataset):
    print(f"\nOriginal Message {i+1}: {message}")

    brute_force_encode = dct_ii(message)
    print("DCT-II (brute):", np.round(brute_force_encode, 4))

    cos2_encode = cos2(message, n)
    print("DCT-II (recursive):", np.round(cos2_encode, 4))

    scipy_encode = dct(message, type=2, norm='ortho')
    print("DCT-II (scipy):   ", np.round(scipy_encode, 4))

    print("Match brute & scipy:", np.allclose(brute_force_encode, scipy_encode))
    print("Match recursive & scipy:", np.allclose(cos2_encode, scipy_encode))
    print("Match brute & recursive:", np.allclose(brute_force_encode, cos2_encode))


Original Message 1: [3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
DCT-II (brute): [10.5     1.2638  0.2355  1.5623 -1.1481 -0.9726 -0.1151  1.1637  0.5
  0.3082  0.1722 -1.0868 -2.7716  0.9964 -1.1839  0.6269]
DCT-II (recursive): [10.5     1.2638  0.2355  1.5623 -1.1481 -0.9726 -0.1151  1.1637  0.5
  0.3082  0.1722 -1.0868 -2.7716  0.9964 -1.1839  0.6269]
DCT-II (scipy):    [10.5     1.2638  0.2355  1.5623 -1.1481 -0.9726 -0.1151  1.1637  0.5
  0.3082  0.1722 -1.0868 -2.7716  0.9964 -1.1839  0.6269]
Match brute & scipy: False
Match recursive & scipy: True
Match brute & recursive: False

Original Message 2: [3 4 0 3 1 4 3 0 0 2 2 1 3 3 2 3]
DCT-II (brute): [ 8.5     0.1772  2.0612 -0.6324 -0.3827  1.6896  1.9494  0.5751 -1.5
 -0.8257  1.8007 -0.7953 -0.9239 -1.7571 -2.0515 -0.9905]
DCT-II (recursive): [ 8.5     0.1772  2.0612 -0.6324 -0.3827  1.6896  1.9494  0.5751 -1.5
 -0.8257  1.8007 -0.7953 -0.9239 -1.7571 -2.0515 -0.9905]
DCT-II (scipy):    [ 8.5     0.1772  2.0612 -0.6324 -0.3827  1.6896  1.

In [9]:
for i, message in enumerate(dataset):
    print(f"\nOriginal Message {i+1}: {message}")

    encoded = cos2(message, n)
    # encoded = dct(message, type=2, norm='ortho')

    brute_decoded = dct_iii(encoded)
    print("DCT-III (brute):", np.round(brute_decoded, 4))

    cos3_decoded = cos3(encoded, n)
    print("DCT-III (cos3):", np.round(cos3_decoded, 4))

    scipy_decoded = dct(encoded, type=3, norm='ortho')
    print("DCT-III (scipy):   ", np.round(scipy_decoded, 4))

    # For TensorFlow version: reshape as batch (1, n)
    tf_input = np.expand_dims(encoded, axis=0)
    DCT3_decoded = DCT3(tf_input, n).numpy()[0]
    print("DCT-III (DCT3):      ", np.round(DCT3_decoded, 4))

    # Compare all
    print("Match brute & scipy:", np.allclose(brute_decoded, scipy_decoded))
    print("Match cos3 & scipy:", np.allclose(cos3_decoded, scipy_decoded))
    print("Match DCT3 & scipy:", np.allclose(DCT3_decoded, scipy_decoded))
    print("Match DCT3 & cos3:", np.allclose(DCT3_decoded, cos3_decoded))


Original Message 1: [3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
DCT-III (brute): [3. 4. 2. 4. 4. 1. 2. 2. 2. 4. 3. 2. 4. 1. 3. 1.]
DCT-III (cos3): [3. 4. 2. 4. 4. 1. 2. 2. 2. 4. 3. 2. 4. 1. 3. 1.]
DCT-III (scipy):    [3. 4. 2. 4. 4. 1. 2. 2. 2. 4. 3. 2. 4. 1. 3. 1.]
DCT-III (DCT3):       [3. 4. 2. 4. 4. 1. 2. 2. 2. 4. 3. 2. 4. 1. 3. 1.]
Match brute & scipy: True
Match cos3 & scipy: True
Match DCT3 & scipy: True
Match DCT3 & cos3: True

Original Message 2: [3 4 0 3 1 4 3 0 0 2 2 1 3 3 2 3]
DCT-III (brute): [3. 4. 0. 3. 1. 4. 3. 0. 0. 2. 2. 1. 3. 3. 2. 3.]
DCT-III (cos3): [ 3.  4.  0.  3.  1.  4.  3.  0. -0.  2.  2.  1.  3.  3.  2.  3.]
DCT-III (scipy):    [ 3.  4. -0.  3.  1.  4.  3.  0.  0.  2.  2.  1.  3.  3.  2.  3.]
DCT-III (DCT3):       [ 3.  4. -0.  3.  1.  4.  3. -0. -0.  2.  2.  1.  3.  3.  2.  3.]
Match brute & scipy: True
Match cos3 & scipy: True
Match DCT3 & scipy: False
Match DCT3 & cos3: False

Original Message 3: [3 0 2 4 2 4 0 1 3 0 3 1 1 0 1 4]
DCT-III (brute): [ 3. -0.  2.  4.  

**Core Reason: Floating Point Precision + TensorFlow Ops**
- cos3 version uses NumPy and native matrix multiplication, which are double-precision (float64) by default and are more numerically stable for small-scale matrix ops.
- DCT3 version uses TensorFlow ops, which default to float32, and includes elementwise operations instead of matrix multiplications, which:
-Reduce computation,
- But also accumulate more rounding errors due to less precise handling in intermediate steps.